In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('../classification_hw11/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
label_encoder = LabelEncoder()
label_encoder.fit(df["home_ownership"])
df["home_ownership"] = label_encoder.transform(df["home_ownership"])
label_encoder.fit(df["verification_status"])
df["verification_status"] = label_encoder.transform(df["verification_status"])
label_encoder.fit(df["issue_d"])
df["issue_d"] = label_encoder.transform(df["issue_d"])
label_encoder.fit(df["initial_list_status"])
df["initial_list_status"] = label_encoder.transform(df["initial_list_status"])
label_encoder.fit(df["pymnt_plan"])
df["pymnt_plan"] = label_encoder.transform(df["pymnt_plan"])
label_encoder.fit(df["next_pymnt_d"])
df["next_pymnt_d"] = label_encoder.transform(df["next_pymnt_d"])
label_encoder.fit(df["application_type"])
df["application_type"] = label_encoder.transform(df["application_type"])
label_encoder.fit(df["hardship_flag"])
df["hardship_flag"] = label_encoder.transform(df["hardship_flag"])
label_encoder.fit(df["debt_settlement_flag"])
df["debt_settlement_flag"] = label_encoder.transform(df["debt_settlement_flag"])

In [7]:
X = df.drop(columns=['loan_status'])
X.fillna(0, inplace=True)
y = df["loan_status"]

In [8]:
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [11]:
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Balanced Random forest classifier

In [12]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [13]:
y_pred_rf = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred_rf)

0.7569918796135928

In [14]:
confusion_matrix(y_test, y_pred_rf)

array([[   64,    37],
       [ 2047, 15057]])

In [15]:
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.63      0.88      0.06      0.75      0.54       101
   low_risk       1.00      0.88      0.63      0.94      0.75      0.57     17104

avg / total       0.99      0.88      0.64      0.93      0.75      0.57     17205



In [16]:
importances = brf.feature_importances_

In [17]:
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.07269927318199497, 'total_rec_prncp'),
 (0.06108290286978353, 'total_pymnt_inv'),
 (0.05799241112202666, 'total_rec_int'),
 (0.05684632497955754, 'total_pymnt'),
 (0.05677675304457769, 'last_pymnt_amnt'),
 (0.03186658561632874, 'int_rate'),
 (0.022622974119503682, 'issue_d'),
 (0.018451411731054752, 'installment'),
 (0.01830326906943508, 'mths_since_recent_inq'),
 (0.017522797954882256, 'dti'),
 (0.017064794663381063, 'out_prncp'),
 (0.016222906823095854, 'out_prncp_inv'),
 (0.016055853321148995, 'max_bal_bc'),
 (0.015537459134109742, 'avg_cur_bal'),
 (0.015376458928054868, 'total_bc_limit'),
 (0.015154663770020577, 'annual_inc'),
 (0.014973494477034586, 'total_bal_ex_mort'),
 (0.01491883566887758, 'tot_hi_cred_lim'),
 (0.014799127773945129, 'mo_sin_old_rev_tl_op'),
 (0.01439257439021563, 'revol_bal'),
 (0.014317480003913589, 'total_bal_il'),
 (0.014289266856132295, 'bc_open_to_buy'),
 (0.014267502159866673, 'mo_sin_old_il_acct'),
 (0.014145406478405738, 'mths_since_rcnt_il'),
 (0.

## EASY ENSEMBLE CLASSIFIER

In [18]:
from imblearn.ensemble import EasyEnsembleClassifier
easy = EasyEnsembleClassifier(n_estimators=100, random_state=1)
easy.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [19]:
y_pred_ea = easy.predict(X_test)
balanced_accuracy_score(y_test, y_pred_ea)

0.9316600714093861

In [20]:
confusion_matrix(y_test, y_pred_ea)

array([[   93,     8],
       [  983, 16121]])

In [21]:
print(classification_report_imbalanced(y_test, y_pred_ea))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205

